# KR1: DLA Serial Implementation

In [1]:
using Plots
using BenchmarkTools
using DelimitedFiles
using Distributions
using GLM
using DataFrames
using Polynomials

include("modules/serialized_dla_modules.jl")
include("modules/time_complexity_module.jl")
include("modules/parallelized_DLA.jl")
include("modules/fractal_dimension_module.jl")

Main.fractal_dimension

## KR1a: `.jl` functions

A set of functions that can generate a single instance of a walker with a random initial position, doing a random walk across the square lattice

### Serial DLA Algorithm

The main function to create a DLA.

In [ ]:
? Random_walker.serialized_dla()

### Pertinent Functions
Functions used in the serial DLA algorithm.

In [ ]:
? Random_walker.initialize_randomwalker()

In [ ]:
? Random_walker.walker_update_position()

In [ ]:
? Random_walker.random_walk_generator()

In [ ]:
? Random_walker.walker_distance_from_cluster()

In [ ]:
? Random_walker.cluster_distance_from_origin()

## KR1b: Implementation

DLA algorithm:
- Generate a set of random walkers in the lattice and a seed position.
- Create a for-loop where for each interaction, only one walker is released and allowed to walk and attach on the cluster.


In [ ]:
particle_number = 1500
maximum_radius = 100.0
sticking_prob = 1.0

@time cluster_aggregate = Random_walker.serialized_dla(particle_number, maximum_radius, sticking_prob);
writedlm("raw_data/cluster aggregate for p = $sticking_prob.txt", cluster_aggregate)

In [ ]:
sticking_prob = 1.0
cluster_aggregate = readdlm("raw_data/cluster aggregate for p = $sticking_prob.txt")

scatter(cluster_aggregate[1, : ], cluster_aggregate[2, :], xlims = (-70, 65), 
            ylims = (-65, 55), size = (700, 700), marker = :diamond )

savefig("image_results/sample cluster aggregate.png")

scatter(cluster_aggregate[1, : ], cluster_aggregate[2, :], xlims = (-70, 65), 
            ylims = (-65, 55), size = (700, 700), marker = :diamond )

## KR1c: Benchmark

Rather than benchmark just the final function `Random_walker.serialized_dla(args)`, we benchmark the pertinent functions it uses.

#### Benchmarking for `initialize_randomwalker` function

In [ ]:
birth_circle = rand(Uniform(1, 10))

@benchmark Random_walker.initialize_randomwalker($birth_circle)

#### Benchmarking for `walker_update_position` function

In [ ]:
birth_circle = rand(Uniform(1, 10))
death_circle = rand(Uniform(10, 15))

@benchmark Random_walker.walker_update_position(Random_walker.initialize_randomwalker($birth_circle), $death_circle, $birth_circle)

#### Benchmarking for `random_walk_generator` function

In [ ]:
step_number = 10000
death_radius = rand(Uniform(10, 15))
birth_radius = rand(Uniform(1, 10))

@benchmark Random_walker.random_walk_generator(step_number, $death_radius, $birth_radius)

#### Benchmarking for `walker_distance_from_cluster` function

In [ ]:
particle_number = 1500
cluster_particle_number = 1490
birth_radius = rand(Uniform(0, 20))
cluster_aggregate = rand(Uniform(0, 10), (2, particle_number))


@benchmark Random_walker.walker_distance_from_cluster($cluster_aggregate, 
        $Random_walker.initialize_randomwalker($birth_radius), cluster_particle_number)

#### Benchmarking for `cluster_distance_from_origin` function

In [ ]:
particle_number = 1500
cluster_particle_number = 1490
cluster_aggregate = rand(Uniform(0, 10), (2, particle_number))

@benchmark Random_walker.cluster_distance_from_origin($cluster_aggregate, cluster_particle_number)

# KR2: DLA Parallelised Implementation

## KR2a: `.jl` functions

A set of functions that generates multiple random walkers that can simultaneously perform a random walk across the square lattice.

## KR2b: Implementation

Parallel versin of the DLA algorithm by releasing multiple particles in the square lattice:
- GPU parallel computing, or
- parallel processing of the random walk trajectories

## KR2c: Benchmark

# KR3: Serial vs Parallelised Implementation

Plot of computational time vs number of random walkers (cluster size).

### Time Complexity of Serial Implementation

Run the following code only if there is no data yet.

In [ ]:
# Generate array and save
# init_particle_number = 5000
# final_particle_number = 5000
# steps = 500
# for i = init_particle_number:steps:final_particle_number
#     particle_number = i
#     serial_time = time_complexity.run_time_serial(particle_number,100.0,1.0)
#     writedlm("raw_data/serial_time particle_number = $particle_number.txt", serial_time)
# end

All the cells below are run for plotting.

In [ ]:
# Compile serial_time array into one
serial_run_time = zeros(10,4)
init_particle_number = 500
final_particle_number = 5000
steps = 500
for i = init_particle_number:steps:final_particle_number
    particle_number = i
    serial_time = readdlm("raw_data/serial_time particle_number = $particle_number.txt")
    j = Int(i/steps)
    serial_run_time[j,:] = serial_time
end

In [ ]:
# Obtain stats array
serial_run_time_stats = time_complexity.run_time_stats(serial_run_time)

In [ ]:
x_stats = serial_run_time_stats[:,1]
y_stats = serial_run_time_stats[:,2]
se = serial_run_time_stats[:,3]
data = DataFrame(x = x_stats, y = y_stats)
model = lm(@formula(y ~ x), data)
intercept, slope = coef(model)

y_best = intercept .+ slope .* x_stats
scatter(x_stats, y_stats, yerror=se, label = "Data", 
    xlabel = "Particle Number", ylabel = "Elapsed Time (s)", 
    title = "Time Complexity (Serial)")
plot!(x_stats, y_best, label = "Best Fit Line", lw = 2)

### Time Complexity of Parallelised Implementation

Run the following code only if there is no data yet.

In [ ]:
# Generate array and save
# init_particle_number = 2000
# final_particle_number = 5000
# steps = 500
# for i = init_particle_number:steps:final_particle_number
#     particle_number = i
#     parallel_time = time_complexity.run_time_parallel(particle_number,0.8,10000,150,300,1)
#     writedlm("raw_data/parallel_time particle_number = $particle_number.txt", parallel_time)
# end

In [ ]:
time1 = @elapsed DLA_parallel.run_parallel_DLA(5000,0.8,10000,150,300,1)

 Mass = 4308, r_B = 149, r_D = 150 

Excessive output truncated after 524321 bytes.

 Mass = 4308, r_B = 149, r_D = 150 

In [ ]:
time2 = @elapsed DLA_parallel.run_parallel_DLA(5000,0.8,10000,150,300,1)

In [ ]:
time3 = @elapsed DLA_parallel.run_parallel_DLA(5000,0.8,10000,150,300,1)

In [ ]:
parallel_time = [5000 time1 time2 time3]
writedlm("raw_data/parallel_time particle_number = 5000.txt", parallel_time)

All the cells below are run for plotting.

In [ ]:
# Compile serial_time array into one
parallel_run_time = zeros(10,4)
init_particle_number = 500
final_particle_number = 5000
steps = 500
for i = init_particle_number:steps:final_particle_number
    particle_number = i
    parallel_time = readdlm("raw_data/parallel_time particle_number = $particle_number.txt")
    j = Int(i/steps)
    parallel_run_time[j,:] = parallel_time
end

In [ ]:
# Obtain stats array
parallel_run_time_stats = time_complexity.run_time_stats(parallel_run_time)

In [ ]:
x_stats = parallel_run_time_stats[:,1]
y_stats = parallel_run_time_stats[:,2]
se = parallel_run_time_stats[:,3]
data = DataFrame(x = x_stats, y = y_stats)
model = lm(@formula(y ~ x), data)
intercept, slope = coef(model)

y_best = intercept .+ slope .* x_stats
scatter(x_stats, y_stats, yerror=se, label = "Data", 
    xlabel = "Particle Number", ylabel = "Elapsed Time (s)", 
    title = "Time Complexity (Parallel)")
plot!(x_stats, y_best, label = "Best Fit Line", lw = 2)

### Time Complexity Log-Log Plot (Serial vs Parallelised)

# KR4: Fractality of Clusters

## KR4a: `.jl` functions

Function for calculating the mass function of the cluster.

In [ ]:
? fractal_dimension.mass_per_radius()

## KR4b: Fractal Dimension

Estimate the fractal dimension.

In [ ]:
matrix = fractal_dimension.mass_per_radius(cluster_aggregate)
x = log.(matrix[:, 1])
y = log.(matrix[:, 2])

data = DataFrame(x = x, y = y)
model = lm(@formula(y ~ x), data)
intercept, slope = coef(model)

scatter(x, y, label = "Data", xlabel = "ln(r)", ylabel = "ln(M(r))", title = "p = $sticking_prob")
plot!(x, intercept .+ slope .* x, label = "Best Fit Line", lw = 2)

savefig("image_results/log-log plot p = $sticking_prob.png")

scatter(x, y, label = "Data", xlabel = "ln(r)", ylabel = "ln(M(r))", title = "p = $sticking_prob")
plot!(x, intercept .+ slope .* x, label = "Best Fit Line", lw = 2)

In [ ]:
println("The slope (fractal dimension) is equal to $slope")